This notebook is used to rip the output scope from a VisionEval "Query-Spec.R" file.

It is not compatible with all possible query specs.

In [1]:
import emat
import os

In [2]:
qs_file = "./VERSPM/Query-Spec.R"

with open(qs_file, 'rt') as qs_f:
    qs = qs_f.read()

# change from R syntax to Python syntax
qs = qs.replace(" <- "," = ")
qs = qs.replace("list(","r_list(")
qs = qs.replace("c(","r_list(")

In [3]:
# R has named list types, Python uses seperate dicts and lists.

def r_list(*args, **kwargs):
    if args and kwargs:
        raise ValueError('all unnamed or named arguments only')
    if args:
        return list(args)
    if kwargs:
        return dict(**kwargs)

In [4]:
exec(qs)

In [5]:
PMSpecifications

[{'Name': 'UrbanHhDvmt_Ma',
  'Summarize': {'Expr': 'sum(UrbanHhDvmt)',
   'Units_': {'UrbanHhDvmt': 'MI/DAY', 'Marea': ''},
   'By': 'Marea',
   'Table': 'Marea'},
  'Units': 'Miles per day',
  'Description': 'Daily vehicle miles traveled by households residing in the urban area'},
 {'Name': 'UrbanHhDvmt_MaAz',
  'Summarize': {'Expr': "sum(Dvmt[Azone == 'RVMPO' & LocType == 'Urban'] )",
   'Units_': {'Dvmt': 'MI/DAY', 'LocType': '', 'Azone': '', 'Marea': ''},
   'By': 'Marea',
   'Table': 'Household'},
  'Units': 'Miles per day',
  'Description': 'Daily vehicle miles traveled by households residing in the urban area in RVMPO county'},
 {'Name': 'UrbanHhDvmt_MaAzMx',
  'Summarize': {'Expr': "sum(Dvmt[Azone == 'RVMPO'& LocType == 'Urban'&  IsUrbanMixNbrhd == '1'])",
   'Units_': {'Dvmt': 'MI/DAY',
    'LocType': '',
    'Azone': '',
    'IsUrbanMixNbrhd': '',
    'Marea': ''},
   'By': 'Marea',
   'Table': 'Household'},
  'Units': 'Miles per day',
  'Description': 'Daily vehicle miles t

In [6]:
pm_scope = emat.Scope(None)

filename = 'Measures_VERSPM_2010,2038_Marea=RVMPO.csv'

def add_measure(*args, **kwargs):
    try:
        pm_scope.add_measure(*args, **kwargs)
    except ValueError:
        pass

for measure in PMSpecifications:
    
    if 'Require' in measure:
        if measure.get('Require',{}).get("Table") == 'Bzone':
            continue
    
    summarize = measure.get('Summarize',{})
    name = measure['Name'].replace("_MaAz","_Az").replace("_Ma","")
    if 'BreakNames' in summarize:
        breaks = list(summarize['BreakNames'].values())[0]
        add_measure(
            f"{name}.min", 
            desc=measure['Description']+f" (up to {breaks[0]})",
            parser=dict(file=filename, loc=[f"{name}.min", 2038]),
        )
        for b0,b1 in zip(breaks[:-1], breaks[1:]):
            add_measure(
                f"{name}.{b0}", 
                desc=measure['Description']+f" ({b0} to {b1})",
                parser=dict(file=filename, loc=[f"{name}.{b0}", 2038]),
            )
        b0 = breaks[-1]
        add_measure(
            f"{name}.{b0}", 
            desc=measure['Description']+f" ({b0} and up)",
            parser=dict(file=filename, loc=[f"{name}.{b0}", 2038]),
        )
    else:
        add_measure(
            name, 
            desc=measure['Description'],
            parser=dict(file=filename, loc=[name, 2038]),
        )

In [7]:
print(pm_scope.dump(indent=4))

scope:
    name: Empty-Scope
    desc: ''
inputs: {}
outputs:
    UrbanHhDvmt:
        kind: info
        desc: Daily vehicle miles traveled by households residing in the urban area
        transform: none
        metamodeltype: linear
        parser:
            file: Measures_VERSPM_2010,2038_Marea=RVMPO.csv
            loc:
            - UrbanHhDvmt
            - 2038
    UrbanHhDvmt_Az:
        kind: info
        desc: Daily vehicle miles traveled by households residing in the urban area
            in RVMPO county
        transform: none
        metamodeltype: linear
        parser:
            file: Measures_VERSPM_2010,2038_Marea=RVMPO.csv
            loc:
            - UrbanHhDvmt_Az
            - 2038
    UrbanHhDvmt_AzMx:
        kind: info
        desc: Daily vehicle miles traveled by households residing in mixed use in
            the urban area in RVMPO county
        transform: none
        metamodeltype: linear
        parser:
            file: Measures_VERSPM_2010,2038_

In [8]:
from emat.model.core_files.parsers import TableParser, MappingParser, loc, key


In [9]:
loc['MareaHouseholdCarSvcDvmt','2038'] - loc['MareaHouseholdCarSvcDvmt','2010']

loc['MareaHouseholdCarSvcDvmt','2038'] - loc['MareaHouseholdCarSvcDvmt','2010']

In [16]:
eval("""(
loc['MareaHouseholdCarSvcDvmt','2038'] 
- loc['MareaHouseholdCarSvcDvmt','2010']
)""")

loc['MareaHouseholdCarSvcDvmt','2038'] - loc['MareaHouseholdCarSvcDvmt','2010']